In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

# Cria a sessão Spark
spark = SparkSession.builder \
    .appName("TestePySpark") \
    .getOrCreate()

In [3]:
import ast

def get_object(path):
    with open(path, "r") as file:
        return file.read()

def generate_tbl_tmp_view(args):
    list_tables_predicates = ast.literal_eval(args["list_tables_predicates"])

    for i in list_tables_predicates:
        db = i.get('database', None)
        tbl = i.get('table', None)
        predicate = i.get('predicate', None)

        if predicate:
            print(f"Lendo {db}.{tbl} com filtro de partição: {predicate}")
            # dyf = glueContext.create_dynamic_frame.from_catalog(
            #     database=db,
            #     table_name=tbl,
            #     push_down_predicate=predicate
            # )
            data = [
                (1, "Alice", 23, 3000),
                (2, "Bob", 30, 4000),
                (3, "Charlie", 25, 3500),
                (4, "David", 30, 5000),
                (5, "Eva", 28, 4500)
            ]

            columns = ["id", "nome", "idade", "salario"]

            df = spark.createDataFrame(data, columns)
        else:
            print(f"Lendo {db}.{tbl} sem filtro de partição")
            # dyf = glueContext.create_dynamic_frame.from_catalog(
            #     database=db,
            #     table_name=tbl
            # )

        # df = dyf.toDF()
        df.createOrReplaceTempView(f"{db}_{tbl}")

In [8]:
args = {
    "JOB_NAME": "glue_job_teste", 
    "SQL_PATH": "db_pedido_venda.tb_cad_fornecedor.sql", 
    "list_tables_predicates": "[{'database': 'db_pedido', 'table': 'tb_cad_prod', 'predicate': 'ano=\"2025\" AND mes=\"08\" AND dia=\"20\"'},{'database': 'db_prod_b', 'table': 'tb_cad_prod_b', 'predicate': 'ano=\"2025\" AND mes=\"08\" AND dia=\"20\"'},{'database': 'db_prod_b', 'table': 'tb_cad_prod_c'}]"
}

# leitura do script SQL no S3
sql_query = get_object(args["SQL_PATH"])

# gera as views temporarias com os predicados passados por argumento
generate_tbl_tmp_view(args)

spark.sql(sql_query).show()

print(f"""
Query:

{sql_query}
""")

Lendo db_pedido.tb_cad_prod com filtro de partição: ano="2025" AND mes="08" AND dia="20"
Lendo db_prod_b.tb_cad_prod_b com filtro de partição: ano="2025" AND mes="08" AND dia="20"
Lendo db_prod_b.tb_cad_prod_c sem filtro de partição
+---+-------+-----+-------+---------+-------+---------+-------+
| id|   nome|idade|salario|nome_do_b|idade_b|nome_do_c|idade_c|
+---+-------+-----+-------+---------+-------+---------+-------+
|  1|  Alice|   23|   3000|    Alice|     23|    Alice|     23|
|  2|    Bob|   30|   4000|      Bob|     30|      Bob|     30|
|  3|Charlie|   25|   3500|  Charlie|     25|  Charlie|     25|
|  4|  David|   30|   5000|    David|     30|    David|     30|
|  5|    Eva|   28|   4500|      Eva|     28|      Eva|     28|
+---+-------+-----+-------+---------+-------+---------+-------+


Query:

-- concatenar {NOME_DATABASE}_{NOME_TABELA}

SELECT 
    a.id,
    a.nome,
    a.idade,
    a.salario,
    b.nome as nome_do_b,
    b.idade as idade_b,
    c.nome as nome_do_c,
    